Then we import the gmsh mesh with the ```dolfinx.io.gmshio``` function.

In [127]:
from mpi4py import MPI
from dolfinx import (
    fem,
    io,
    default_scalar_type,
    geometry,
    __version__ as dolfinx_version,
)
from dolfinx.fem.petsc import LinearProblem
import ufl
import numpy as np
import numpy.typing as npt
from packaging.version import Version


v_bc_wall_tag = 31
v_bc_inlet_tag = 29
v_bc_outlet_tag = 30
mesh_data = io.gmshio.read_from_msh("double_slit_mesh.msh", MPI.COMM_WORLD, 0, gdim=2)
if Version(dolfinx_version) > Version("0.9.0"):
    domain = mesh_data.mesh
    assert mesh_data.facet_tags is not None
    facet_tags = mesh_data.facet_tags
else:
    domain, boundary_tags, facet_tags = mesh_data

domain.topology.create_connectivity(domain.topology.dim - 1, domain.topology.dim)

# domain.topology.dim
boundary_tags.find(v_bc_outlet_tag)

Info    : Reading 'double_slit_mesh.msh'...
Info    : 33 entities
Info    : 20676 nodes
Info    : 41352 elements
Info    : Done reading 'double_slit_mesh.msh'


array([], dtype=int32)

We define the function space for our unknown $p$ and define the range of frequencies we want to solve the Helmholtz equation for.

In [128]:

V = fem.functionspace(domain, ("Lagrange", 2))

# Air parameters
omega = 6 * np.pi * 1000  # Angular frequency (rad/s)
rho0 = 1.225  # kg/m^3
c = 340  # m/s
k = omega / c  # Wave number (1/m)
v_n = np.exp(1j * omega * 0)  # Normal velocity amplitude at inlet (m/s)

We also need to specify the integration measure $ds$, by using ```ufl```, and its built in integration measures

In [129]:
ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tags)

## Variational Formulation
We can now write the variational formulation.

In [130]:
p = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

a = (
    ufl.inner(ufl.grad(p), ufl.grad(v)) * ufl.dx
    # + 1j * rho0 * omega / Z * ufl.inner(p, v) * ds(v_bc_inlet_tag)
    - k**2 * ufl.inner(p, v) * ufl.dx
)
L = -1j * omega * rho0 * ufl.inner(v_n, v) *ds(v_bc_inlet_tag)

The class ```LinearProblem``` is used to setup the PETSc backend and assemble the system vector and matrices.
The solution will be stored in a `dolfinx.fem.Function`, ```p_a```.

In [131]:
bc_inlet = fem.dirichletbc(v_n, fem.locate_dofs_topological(V, domain.topology.dim - 1, facet_tags.find(v_bc_inlet_tag)), V)
bc_wall = fem.dirichletbc(0.0 + 0j, fem.locate_dofs_topological(V, domain.topology.dim - 1, facet_tags.find(v_bc_wall_tag)), V)

# fem.locate_dofs_topological(V, domain.topology.dim - 1, facet_tags.find(v_bc_inlet_tag))
# fem.locate_dofs_topological(V, domain.topology.dim - 1, facet_tags.find(v_bc_wall_tag))

In [132]:
p_a = fem.Function(V)
p_a.name = "pressure"

problem = LinearProblem(
    a,
    L,
    u=p_a,
    petsc_options={
        "ksp_type": "preonly",
        "pc_type": "lu",
        # "pc_factor_mat_solver_type": "mumps",
    },
    bcs=[bc_inlet,bc_wall],
)

In [133]:

problem.solve()
p_a.x.scatter_forward()

In [134]:
p_a.x.array.max()

np.complex128(2.4816024076270504+0j)

In [135]:
with io.VTKFile(domain.comm, "solution.pvd", "w") as file:
    file.write_mesh(domain)
    file.write_function([p_a])